# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [ ]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [7]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
#from fbprophet import Prophet
#from fbprophet.plot import plot_plotly, plot_components_plotly
from prophet import Prophet
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [6]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-05-03'
start_date = '2016-01-01'
# Descargar el dataframe
btc_df = yf.download('BTC-USD',start_date, today).reset_index()

btc_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
2674,2023-04-28,"$29,481.01","$29,572.79","$28,929.61","$29,340.26","$29,340.26",17544464887
2675,2023-04-29,"$29,336.57","$29,452.46","$29,088.04","$29,248.49","$29,248.49",10662634333
2676,2023-04-30,"$29,245.52","$29,952.03","$29,114.02","$29,268.81","$29,268.81",14652199272
2677,2023-05-01,"$29,227.10","$29,329.94","$27,680.79","$28,091.57","$28,091.57",18655599976
2678,2023-05-02,"$28,087.18","$28,881.30","$27,924.12","$28,680.54","$28,680.54",16432924527


In [9]:
# Validamos que no hay datos vacíos
btc_df.isnull().sum()   #df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [14]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = btc_df[['Date', 'Open']]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df = df.rename(columns=new_names) #De esta manera renombramos las columnas del
#antiguo dataframe por los nuevos nombres definidos arriba en este mismo script

In [15]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [16]:
df.tail()

,ds,y
2674,2023-04-28,"$29,481.01"
2675,2023-04-29,"$29,336.57"
2676,2023-04-30,"$29,245.52"
2677,2023-05-01,"$29,227.10"
2678,2023-05-02,"$28,087.18"


In [17]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = ""
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [34]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
#m = Prophet()   #Inicializamos el modelo con la variable "m", el Prophet se inicio
#en el punto 1 de este collab
m=Prophet(seasonality_mode = 'multiplicative')    #Hiperparámetro

In [ ]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)   #df es el nombre de nuestro dataframe

<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [41]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(365)    #Nombre de nuestra variable = m, 365 =
#= es el número de dias que queremos predecir
future.tail()

,ds
3039,2024-04-27
3040,2024-04-28
3041,2024-04-29
3042,2024-04-30
3043,2024-05-01


In [42]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-01
1,2016-01-02
2,2016-01-03
3,2016-01-04
4,2016-01-05
...,...
3039,2024-04-27
3040,2024-04-28
3041,2024-04-29
3042,2024-04-30


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [43]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [44]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
3039,2024-04-27,"$-10,659.53","$-29,478.39","$8,321.98"
3040,2024-04-28,"$-10,677.25","$-29,050.45","$7,931.86"
3041,2024-04-29,"$-10,687.41","$-28,720.50","$8,573.88"
3042,2024-04-30,"$-10,682.99","$-28,275.29","$8,442.02"
3043,2024-05-01,"$-10,647.73","$-29,027.48","$7,873.83"


In [47]:
from prophet.plot import plot_plotly, plot_components_plotly

<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [48]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [49]:
plot_components_plotly(m, forecast)